### решить задачу классификации на предложенном датасете с использованием Boosting моделей.
Датасет: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009


# IMPORTS

In [ ]:
import pandas as pd
from os import path

import xgboost as xgb
from xgboost.sklearn import XGBClassifier, XGBRegressor

from sklearn import metrics
from sklearn.model_selection import cross_val_score,  train_test_split

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# MOUNT DRIVE

In [ ]:
# Mount gdrive
from google.colab import drive
drive.mount('/content/gdrive')

ROOT_PATH = f"/content/gdrive/My Drive/temp/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#Loading dataset
filename = "winequality-red.csv"
wine = pd.read_csv(path.join(ROOT_PATH, filename))

In [ ]:
wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
wine['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [ ]:
wine.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

We have no gapes.

# PREPROCESSING

In [ ]:
X = wine.drop(["quality"], axis=1)
y = wine["quality"]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [ ]:
X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled.shape

(1599, 11)

In [ ]:
X_scaled[10:20,:]

array([[-0.93053092,  0.29149948, -0.98066858, -0.5241662 ,  0.20262147,
        -0.08366945,  0.56354529, -0.44875156, -0.20159105, -0.69723268,
        -1.1479806 ],
       [-0.47090657, -0.15541864,  0.45714436,  2.5265889 , -0.3499753 ,
         0.10759209,  1.68867708,  0.55827446,  0.25195842,  0.83710732,
         0.07229363],
       [-1.56251441,  0.48702616, -1.39147228, -0.66606179,  0.0325917 ,
         0.01196132,  0.38109149, -1.29677347,  1.74219238, -0.81525883,
        -0.49090986],
       [-0.29854743,  0.45909378,  0.09769112, -0.66606179,  0.56393475,
        -0.65745405, -0.53117752,  0.34626898, -0.33117661,  5.32210117,
        -1.24184785],
       [ 0.33343606,  0.51495855, -0.46716396,  0.89478966,  1.88166551,
         3.45466896,  2.99626267,  0.98228542, -0.97910442,  1.30921194,
        -1.1479806 ],
       [ 0.33343606,  0.51495855, -0.4158135 ,  0.96573745,  1.75414318,
         3.35903819,  3.08748957,  0.98228542, -0.91431164,  1.60427732,
        -1.147

# train/test split

In [ ]:
train_test_split?

In [ ]:
from sklearn.model_selection import train_test_split

# TODO: сдлелать stratify после upsampling'а
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42784)

# models

In [ ]:
def fit_model(model, X, y, X_test, y_test, parameters, scoring="accuracy"):  
  cross_validation = StratifiedKFold(n_splits=5)

  grid_search = GridSearchCV(model,
                              scoring=scoring,
                              param_grid=parameters,
                              cv=cross_validation,
                              verbose=1
                            )

  grid_search.fit(X, y)
  parameters=grid_search.best_params_
  print('Best score: {}'.format(grid_search.best_score_))
  print('Best parameters: {}'.format(parameters))

  return grid_search

## XGBClassifier

In [ ]:
XGBClassifier?

```
Init signature: XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, verbosity=1, silent=None, objective='binary:logistic', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, colsample_bynode=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, **kwargs)
Docstring:     
Implementation of the scikit-learn API for XGBoost classification.

Parameters
----------
max_depth : int
    Maximum tree depth for base learners.
learning_rate : float
    Boosting learning rate (xgb's "eta")
n_estimators : int
    Number of trees to fit.
verbosity : int
    The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
silent : boolean
    Whether to print messages while running boosting. Deprecated. Use verbosity instead.
objective : string or callable
    Specify the learning task and the corresponding learning objective or
    a custom objective function to be used (see note below).
booster: string
    Specify which booster to use: gbtree, gblinear or dart.
nthread : int
    Number of parallel threads used to run xgboost.  (Deprecated, please use ``n_jobs``)
n_jobs : int
    Number of parallel threads used to run xgboost.  (replaces ``nthread``)
gamma : float
    Minimum loss reduction required to make a further partition on a leaf node of the tree.
min_child_weight : int
    Minimum sum of instance weight(hessian) needed in a child.
max_delta_step : int
    Maximum delta step we allow each tree's weight estimation to be.
subsample : float
    Subsample ratio of the training instance.
colsample_bytree : float
    Subsample ratio of columns when constructing each tree.
colsample_bylevel : float
    Subsample ratio of columns for each level.
colsample_bynode : float
    Subsample ratio of columns for each split.
reg_alpha : float (xgb's alpha)
    L1 regularization term on weights
reg_lambda : float (xgb's lambda)
    L2 regularization term on weights
scale_pos_weight : float
    Balancing of positive and negative weights.
base_score:
    The initial prediction score of all instances, global bias.
seed : int
    Random number seed.  (Deprecated, please use random_state)
random_state : int
    Random number seed.  (replaces seed)
missing : float, optional
    Value in the data which needs to be present as a missing value. If
    None, defaults to np.nan.
importance_type: string, default "gain"
    The feature importance type for the feature_importances_ property: either "gain",
    "weight", "cover", "total_gain" or "total_cover".
\*\*kwargs : dict, optional
    Keyword arguments for XGBoost Booster object.  Full documentation of parameters can
    be found here: https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst.
    Attempting to set a parameter via the constructor args and \*\*kwargs dict simultaneously
    will result in a TypeError.

    .. note:: \*\*kwargs unsupported by scikit-learn

        \*\*kwargs is unsupported by scikit-learn.  We do not guarantee that parameters
        passed via this argument will interact properly with scikit-learn.

Note
----
A custom objective function can be provided for the ``objective``
parameter. In this case, it should have the signature
``objective(y_true, y_pred) -> grad, hess``:

y_true: array_like of shape [n_samples]
    The target values
y_pred: array_like of shape [n_samples]
    The predicted values

grad: array_like of shape [n_samples]
    The value of the gradient for each sample point.
hess: array_like of shape [n_samples]
    The value of the second derivative for each sample point
File:           /usr/local/lib/python3.6/dist-packages/xgboost/sklearn.py
Type:           type
```

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder.fit(y)

array([2, 2, 2, ..., 3, 2, 3])

In [ ]:
y_train_labeled = label_encoder.transform(y_train)
y_test_labeled = label_encoder.transform(y_test)

In [ ]:
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=2,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'multi:softmax',
        nthread=4,
        scale_pos_weight=1,
        seed=27)

cv_parameters = {
  "n_estimators": [2,3,5,10,20],
  "max_depth": range(2,10+1),
  'gamma': [i/10.0 for i in range(0,5)]
}

trained = fit_model(xgb1, X_train, y_train_labeled, X_test, y_test_labeled, parameters=cv_parameters, scoring="accuracy")

Fitting 5 folds for each of 225 candidates, totalling 1125 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1125 out of 1125 | elapsed:  1.5min finished


Best score: 0.6613844870725605
Best parameters: {'gamma': 0.1, 'max_depth': 10, 'n_estimators': 20}


In [ ]:
#Predict training set:
dtrain_predictions = trained.predict(X_train)
dtrain_predprob = trained.predict_proba(X_train)[:,1]
    
#Print model report:
print ("\nModel Report")
print ("Accuracy (Train): %.4g" % metrics.accuracy_score(y_train_labeled, dtrain_predictions))
# print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train_labeled, dtrain_predprob))

# Predict on testing data:
dtest_predictions = trained.predict(X_test)
dtest_predprob = trained.predict_proba(X_test)[:,1]
print ("Accuracy (Test): %.4g" % metrics.accuracy_score(y_test_labeled, dtest_predictions))
#print ('AUC Score (Test): %f' % metrics.roc_auc_score(y_test_labeled, dtest_predprob))


Model Report
Accuracy (Train): 0.9508
Accuracy (Test): 0.68
